## [Stable Marriage](https://www.cs.princeton.edu/courses/archive/spr05/cos423/lectures/01stable-matching.pdf)
### Problem
- Give:
    + N men
    + N women
- Each person has ranked all members of the opposite sex in **order of preference**. Example:
```python
# Men
    0: 8, 6, 7, 5, 9
    1: 9, 7, 6, 5, 8
    2: 6, 9, 5, 8, 7
    3: 9, 6, 8, 7, 5
    4: 8, 5, 6, 7, 9
# Women
    5: 3, 1, 4, 2, 0
    6: 1, 0, 3, 2, 4
    7: 0, 2, 4, 3, 1
    8: 3, 0, 2, 1, 4
    9: 1, 4, 0, 2, 3
```

- Find a **Stable Matching** = Perfect matching + Stability:
    - **Perfect matching**
        + Each man gets exactly one woman.
        + Each woman gets exactly one man.
    - **Stability**
        - no unstable pairs.
        - **Unstable pair** =
            + Exist a pair (m_, w_) which m_ **not marry** w_
            + m_ prefer w_ **and** w_ prefer m_ than their current partners

        - Example:
        ```python
        # Men
            0: 2, 3
            1: 2, 3
        # Women
            2: 0, 1
            3: 0, 1
        ```
            - Match {0,3}, {1,2} 
                + 0 prefer 2 over 3 and 2 prefer 0 over 1 -> **not stable**
            - Match {0,2}, {1,3} -> **stable**

### Gale–Shapley algorithm O($n^2$)
- A stable matching will **always exist**
```python
while exist_a_free_man():
    m = a_free_man()
    w = m_highest_reference()

    while m_not_yet_engaged():
        # Case: w is free
        if w_is_free():
            engage(m, w)
         
        # Case: w is already engaged with m*
        #       w prefer m than m*
        else if w_prefers_m_than_m*():
            divorce(m*, w)
            set_free(m*)

            engage(m, w)

        # Case: w is already engaged with m*
        #       w prefer m* than m
        else:
            w = next_m_preference()
```

### Time Complexity Analysis
- There are at most $n^2$ possible proposals

### Proof of Correctness
#### Termination
- Men propose to women in decreasing order of preference.
- Once a woman is matched, she never becomes unmatched, she only "trades up".

#### Perfection
- All men and women get engaged
- Proof by contradiction
    + Suppose a man m not engaged upon the termination of algorithm.
    + -> a woman w not engaged
    + -> w never got proposed (if a woman is proposed -> alway be engaged)
    + -> But m proposes to every woman if not engaged
    + -> Contrast

#### Stability
- No unstable pairs.
- Proof by contradiction
    + Suppose w* prefer m* and m* prefer w* over their current partners
    + Case 1: m* not propose to w* -> but m* propose in his priority -> m* prefer his current partner than w* -> constrast
    + Case 2: m* propose to w* -> but w* engage her partner -> w* prefer her partner than m* -> Contrast

## Code
```C++
// preference: 
//      [0:N-1]: men[N]
//      [N:2N-1]: women[N]
vector<int> stableMarriage(const vector<vector<int>> &preference) {
    // N = number of men/women
    int N = preference.size() / 2;

    // man, woman and the other guy
    int m, w, m_;

    // engaged[m] = w, engaged[w] = m
    vector<int> engaged(2*N, -1);

    // Free man queue
    queue<int> freeMan;
    for(m=0; m<N; ++m) freeMan.push(m);

    // Prebuild wPrefer[w][m][m_]: Check if w prefer m over m_
    vector<vector<vector<bool>>> wPrefer(
        2*N,
        vector<vector<bool>>(N,
        vector<bool>(N, false)));
    for(w=N; w<2*N; ++w) {
        for(int i=0; i<N-1; ++i) for(int j=i+1; j<N; ++j) {
            wPrefer[w][preference[w][i]][preference[w][j]] = true;
        }
    }

    // Matching
    while(freeMan.empty() == false) {
        // Pick a free man m
        m = freeMan.front();
        freeMan.pop();

        // Pick m highest preference
        int i = 0;

        while(engaged[m] == -1) {
            w = preference[m][i];
            m_ = engaged[w];

            // Case: w is free
            if(engaged[w] == -1) {
                // Engage(m,w)
                engaged[w] = m;
                engaged[m] = w;
            }

            // Case: w is already engaged with m_
            //       w prefer m than m_
            else if(wPrefer[w][m][m_]) {
                // Divorce(m_, w) and set free m_
                engaged[m_] = -1;
                freeMan.push(m_);

                // Engage(m,w)
                engaged[w] = m;
                engaged[m] = w;
            }

            // Case: w is already engaged with m_
            //       w prefer m_ than m
            else ++i;   
        }
    }

    return engaged;
}
```

####  Testcase 
- Test case 1
    - Input
    ```
    2
    2 3
    2 3
    0 1
    0 1
    ```

    - output
    ```
    [0]2 [1]3 [2]0 [3]1
    ```
- Test case 2
    - Input
    ```
    5
    8 6 7 5 9
    9 7 6 5 8
    6 9 5 8 7
    9 6 8 7 5
    8 5 6 7 9
    3 1 4 2 0
    1 0 3 2 4
    0 2 4 3 1
    3 0 2 1 4
    1 4 0 2 3
    ```

    - output
    ```
    [0]8 [1]9 [2]7 [3]6 [4]5 [5]4 [6]3 [7]2 [8]0 [9]1
    ```
- Test case 3
    - Input
    ```
    4
    7 5 6 4
    5 4 6 7
    4 5 6 7
    4 5 6 7
    0 1 2 3
    0 1 2 3
    0 1 2 3
    0 1 2 3
    ```

    - output
    ```
    [0]7 [1]5 [2]4 [3]6 [4]2 [5]1 [6]3 [7]0
    ```
